<a href="https://www.kaggle.com/code/quanghiu0711/funie-gan?scriptVersionId=284944538" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

In [1]:
import os

for root, dirs, files in os.walk('/kaggle/input', topdown=False):
    for name in files:
        print(os.path.join(root, name))


/kaggle/input/euvp-dataset/EUVP/eval_data/ufo_test/funie-gan-up/funie-gan-up/set_f48.jpg
/kaggle/input/euvp-dataset/EUVP/eval_data/ufo_test/funie-gan-up/funie-gan-up/set_u24.jpg
/kaggle/input/euvp-dataset/EUVP/eval_data/ufo_test/funie-gan-up/funie-gan-up/set_u41.jpg
/kaggle/input/euvp-dataset/EUVP/eval_data/ufo_test/funie-gan-up/funie-gan-up/set_f32.jpg
/kaggle/input/euvp-dataset/EUVP/eval_data/ufo_test/funie-gan-up/funie-gan-up/set_f9.jpg
/kaggle/input/euvp-dataset/EUVP/eval_data/ufo_test/funie-gan-up/funie-gan-up/set_u4.jpg
/kaggle/input/euvp-dataset/EUVP/eval_data/ufo_test/funie-gan-up/funie-gan-up/set_u47.jpg
/kaggle/input/euvp-dataset/EUVP/eval_data/ufo_test/funie-gan-up/funie-gan-up/set_f29.jpg
/kaggle/input/euvp-dataset/EUVP/eval_data/ufo_test/funie-gan-up/funie-gan-up/set_u25.jpg
/kaggle/input/euvp-dataset/EUVP/eval_data/ufo_test/funie-gan-up/funie-gan-up/set_u30.jpg
/kaggle/input/euvp-dataset/EUVP/eval_data/ufo_test/funie-gan-up/funie-gan-up/set_f44.jpg
/kaggle/input/euvp-data

In [2]:
# =============================
# FUNIE-GAN PyTorch - Batch Enhancer
# =============================
import torch
import torchvision.transforms as T
from PIL import Image
import matplotlib.pyplot as plt
import numpy as np
import os, sys, glob
from tqdm import tqdm
from pathlib import Path

print("Using PyTorch:", torch.__version__)
print("GPU Available:", torch.cuda.is_available())

device = "cuda" if torch.cuda.is_available() else "cpu"

# =============================
# 1. AUTO FIND funiegan.py
# =============================
search = glob.glob("/kaggle/input/funie-gan/**/funiegan.py", recursive=True)
if len(search) == 0:
    raise Exception("Không tìm thấy funiegan.py!")

funie_path = os.path.dirname(search[0])
sys.path.append(funie_path)

print("FUNIE-GAN PyTorch net path:", funie_path)

from funiegan import GeneratorFunieGAN

# =============================
# 2. LOAD WEIGHTS (.pth)
# =============================
weights = glob.glob("/kaggle/input/funie-gan/**/funie_generator.pth", recursive=True)
if len(weights) == 0:
    raise Exception("Không tìm thấy funie_generator.pth!")

weight_path = weights[0]
print("Using weights:", weight_path)

netG = GeneratorFunieGAN().to(device)
netG.load_state_dict(torch.load(weight_path, map_location=device))
netG.eval()
print("Generator loaded successfully!")

# =============================
# 3. IMAGE TRANSFORMS
# =============================
transform = T.Compose([
    T.Resize((256,256)),
    T.ToTensor()
])

def enhance_image(path):
    img = Image.open(path).convert("RGB")
    inp = transform(img).unsqueeze(0).to(device)
    with torch.no_grad():
        out = netG(inp)[0].cpu()
    out = (out.permute(1,2,0).numpy() * 255).astype(np.uint8)
    return np.array(img), out

# =============================
# 4. FIND INPUT FOLDER
# =============================
possible_dirs = [
    "/kaggle/input/euvp-dataset/input/",
    "/kaggle/input/euvp-dataset/EUVP/test/",
    "/kaggle/input/euvp-dataset/"
]

input_dir = None
for d in possible_dirs:
    if os.path.isdir(d):
        input_dir = d
        break

if input_dir is None:
    raise Exception("Không tìm thấy thư mục ảnh EUVP.")

print("Input dir:", input_dir)

# Scan ảnh
image_paths = []
for ext in ("*.jpg","*.png","*.jpeg"):
    image_paths.extend(glob.glob(os.path.join(input_dir, "**", ext), recursive=True))

image_paths = sorted(list(set(image_paths)))
print("Found images:", len(image_paths))

if len(image_paths) == 0:
    raise Exception("Không có ảnh nào trong folder!")

# =============================
# 5. OUTPUT DIR
# =============================
OUT_DIR = "/kaggle/working/funie_results/"
os.makedirs(OUT_DIR, exist_ok=True)

# =============================
# 6. BATCH ENHANCEMENT
# =============================
print("Enhancing images...")

for img_path in tqdm(image_paths):
    orig, enhanced = enhance_image(img_path)
    name = os.path.basename(img_path)
    save_path = os.path.join(OUT_DIR, name)
    Image.fromarray(enhanced).save(save_path)

print("DONE! Saved to:", OUT_DIR)



Using PyTorch: 2.6.0+cu124
GPU Available: True
FUNIE-GAN PyTorch net path: /kaggle/input/funie-gan/tensorflow2/default/1/PyTorch/nets
Using weights: /kaggle/input/funie-gan/tensorflow2/default/1/PyTorch/models/funie_generator.pth
Generator loaded successfully!
Input dir: /kaggle/input/euvp-dataset/
Found images: 30658
Enhancing images...


100%|██████████| 30658/30658 [07:19<00:00, 69.74it/s]

DONE! Saved to: /kaggle/working/funie_results/
